In [ ]:
import pandas as pd
import numpy as np
import itertools 
from fuzzywuzzy import fuzz

def checkString(str1,str2):
    #token_set_ratio ignores upper and lower case letters , token_sort_ratio ignore order of letters
    #if(fuzz.token_set_ratio(str1,str2) > 80):
    #WRatio handles lower and upper cases and some other parameters too
    if(fuzz.WRatio(str1,str2) > 80):
        return True
    else:
        return False
        
#traversing routes.txt file to get route_id corresponding to the route name
def routes_txt(route_lname):
    t  = route_lname+"_DTC"
    df1 = pd.read_csv('/home/anjali/Desktop/DTC_CHANGES/routes.txt',delimiter = ",",dtype={'route_short_name':'float64','route_long_name':'object','route_type':'int','route_id':'int','agency_id':'object'})
    if t in df1.values:
        routeid = df1[(df1['route_long_name'] == t) & (df1['agency_id']=='DTC')]['route_id'].item()
        k = trip_txt(routeid)
        return k
    else:
        return []

#traversing trip.txt file to fing trip id corresponding to the route_id
def trip_txt(routeid):
    df2 = pd.read_csv('/home/anjali/Desktop/DTC_CHANGES/trips.txt', delimiter = ",",dtype = {'route_id': 'int64', 'service_id': 'int64', 'trip_id': 'O', 'shape_id': 'float64'})
    tripid = df2.loc[(df2['route_id']==routeid),'trip_id'].unique()[0]
    z = stop_times_txt(tripid)
    return z

#traversing stop_times.txt file to find count all stop counts correspoding to the trip_id
def stop_times_txt(tripid):
    old_stops = []
    df3 = pd.read_csv('/home/anjali/Desktop/DTC_CHANGES/stop_times.txt',delimiter = ",",dtype = {'trip_id': 'O', 'arrival_time': 'O', 'departure_time': 'O', 'stop_id': 'O', 'stop_sequence': 'int'})
    stopid = df3.loc[df3['trip_id']==tripid,'stop_id'].tolist()
    for x in stopid:
        p = stop_txt(x)
        old_stops.append(p)      
    return old_stops

#traversing stop.txt file to find all stop names correspoding to the trip_id
def stop_txt(stopid):
    df4 = pd.read_csv('/home/anjali/Desktop/DTC_CHANGES/stops.txt', delimiter=",")
    stopname = df4.loc[df4['stop_id']== stopid,'stop_name'].tolist()
    return stopname[0]
    

#function to match all stops sequences in old and new data for all routes
def MatchDataSequence():
    #extracting names of all routes in New DTC Data from DTC_NewData txt file
    df = pd.read_csv('/home/anjali/Desktop/DTC_CHANGES/DTC_NewData.txt', delimiter = ",")
    for ind in df.index:
            filenames = df['DTC_routes'][ind]
            fileName , separator , extension = filenames.partition('.')
            
            #passing the route name to routes_txt , storing old data stops in a list lst1 
            lst1 = routes_txt(fileName)#list of old stops
            
            #reading new dtc data from .csv files
            df7 = pd.read_csv(f'/home/anjali/Desktop/NewDTCdata/{filenames}',delimiter = ",")
            
            #sto
            if('Stops_up' in df7.columns):
                lst2 = df7['Stops_up'].tolist()#list of new stops
            else:
                lst2 = df7['Stops_down'].tolist()#list of new stops
            
            #string matching for old and new stops
            print(f"Route : {fileName}")
            print(f"Total stops in old data : {len(lst1)}")
            print(f"Total stops in new data : {len(lst2)}")
            if(len(lst1) and len(lst2)):
                for (a, b) in itertools.zip_longest(lst1,lst2):
                    str1 = a
                    str2 = b
                    #print(f"old : {str1} , new:  {str2}")
                    if(checkString(str1,str2) == False):
                        print(f"Stop did not match : {str1} & {str2}")
                        
            else:
                if len(lst1) == 0:
                    print("No stops in Old data")
                else:
                    print("No stops in New data")
                    
MatchDataSequence()

Route : 0114(NS)DOWN
Total stops in old data : 0
Total stops in new data : 68
No stops in Old data
Route : 0114(NS)UP
Total stops in old data : 69
Total stops in new data : 69
Stop did not match : Shalimar Bagh Richi Rich & Shalimar Bagh Xing
Stop did not match : Mangolpur School & Mangol Puri  Police Line
Stop did not match : Mangolpur Kalan Village & Mangol Pur Kalan
Stop did not match : Radha Swami Satsang & Rupali Enclave RS Satsang
Stop did not match : Ladpur Village & Lad Pur
Route : 0118EXT(NS)UP
Total stops in old data : 39
Total stops in new data : 39
Stop did not match : Kondali Xing & Kondli
Stop did not match : Shashi Garden Xing & Shashi Garden More
Stop did not match : Patpar Ganj Xing & Samas Pur Village
Stop did not match : MCKR Hospital & Shakkar pur School Block
Stop did not match : Defence Colony & Laxmi Nagar Xing
Stop did not match : Delhi Secretariat & Rainy Well
Stop did not match : ITO Ring Road & Delhi Secretariat
Route : 012(NS)UP
Total stops in old data : 93


Route : 072(NS)UP
Total stops in old data : 69
Total stops in new data : 69
Stop did not match : Dholi Piao & District Centre Janak Puri
Stop did not match : Nawada Gaon & Navada Village
Route : 0729(NS)DOWN
Total stops in old data : 45
Total stops in new data : 45
Stop did not match : Samalkha Crossing & Samalkha Xing
Stop did not match : MCD Store (Rajpur Road) & Shiv Murti
Stop did not match : Aruna Asaf Ali Hospital & Rang Puri
Stop did not match : Bakhtawar Pur School & Shivaji Park
Route : 0729(NS)UP
Total stops in old data : 46
Total stops in new data : 46
Stop did not match : Bakhtawar Pur School & Shivaji Park
Stop did not match : Tibiya College & P.S. Parliament
Stop did not match : APS Colony & Jharera Village
Stop did not match : Aruna Asaf Ali Hospital & Rang Puri
Stop did not match : MCD Store (Rajpur Road) & Shiv Murti
Stop did not match : Rajpur Road Transport Deptt & Rajokari Xing
Stop did not match : PNB STA Road & Shiv Murti-II
Stop did not match : Samalkha Crossing 

Route : 0901(NS)DOWN
Total stops in old data : 56
Total stops in new data : 56
Stop did not match : Mangolpur Kalan Village & Mangol Pur Kalan
Stop did not match : Mangolpur School & Mangol Puri Police Line
Stop did not match : Shalimar Bagh Richi Rich & Shalimar Bagh Xing
Stop did not match : Dichau Kalan Depot & Kamla Market
Stop did not match : Prem Nagar & New Delhi Railway Station Gate 2
Route : 0901(NS)UP
Total stops in old data : 60
Total stops in new data : 60
Stop did not match : Prem Nagar & New Delhi Railway Station Gate 2
Stop did not match : Ajmeri Gate / Kamla Market & Vivekanand Marg
Stop did not match : Shalimar Bagh Richi Rich & Shalimar Bagh Xing
Stop did not match : Mangolpur School & Mangol Puri Police Line
Stop did not match : Mangolpur Kalan Village & Mangol Pur Kalan
Route : 0926(NS)DOWN
Total stops in old data : 58
Total stops in new data : 58
Stop did not match : Hiran Kudna Crossing & Hiran Kudna More
Stop did not match : Lokesh Cinema Nangloi & Nangloi JJ Col

Route : 103DOWN
Total stops in old data : 49
Total stops in new data : 49
Stop did not match : Munim Ji Ka Bagh & Muneem Ka Bagh
Stop did not match : Minda Farm & Arvind Farm
Stop did not match : Alipur Village GT Road & Ali Pur GT Road
Stop did not match : GTK Bypass / Mukraba Chowk & G.T.K By Pass
Stop did not match : Jahangir Puri Outer Ring Road & Jahangir Puri Crossing
Route : 103STLDOWN
Total stops in old data : 27
Total stops in new data : 27
Stop did not match : Munim Ji Ka Bagh & Muneem Ka Bagh
Stop did not match : Minda Farm & Arvind Farm
Stop did not match : Alipur Village GT Road & Ali Pur GT Road
Stop did not match : GTK Bypass / Mukraba Chowk & G.T.K By Pass
Stop did not match : Jahangir Puri Outer Ring Road & Jahangir Puri Crossing
Route : 103STLUP
Total stops in old data : 35
Total stops in new data : 35
Stop did not match : Sabji Mandi Azadpur & Azad Pur New Subzi Mandi
Stop did not match : Jahangirpuri GT Road & Jahangripuri Metro station
Stop did not match : Jahangir

Route : 107UP
Total stops in old data : 63
Total stops in new data : 65
Stop did not match : Sabji Mandi Azadpur & Azad Pur New Subzi Mandi
Stop did not match : Prem Nagar & Sarai Peepal Thala
Stop did not match : Jahangir Puri Metro Station & Jahangir Puri Crossing
Stop did not match : GTK Bypass / Mukraba Chowk & G.T.K By Pass
Stop did not match : Badli Crossing & Badli Xing
Stop did not match : Badli Factory Area & Rohini Depot 4
Stop did not match : Rohini Sec 18 Block & Sardar Colony
Stop did not match : Shahbad Daulatpur School & St Xavier School Daulatpur
Stop did not match : Barwala School & Barwala Pathshala
Stop did not match : Pooth Khurd Village & Dharamshala Pooth Khurd
Stop did not match : DHARAMSHALA POOTH KHURD & Pooth Khurd village
Stop did not match : Pooth Khurd Hospital & Mahrishi Balmiki Hospital
Stop did not match : Bawana School & DSIDC Sec-1(Bawana)
Stop did not match : Bawana Village & DSIDC Office Bawana
Stop did not match : Bawana Depot & Bawana School
Stop d

...

Route : 234DOWN
Total stops in old data : 46
Total stops in new data : 46
Stop did not match : Yamuna Vihar Brijpuri & C Block Yamuna Vihar
Stop did not match : Khajuri & Khazoori
Route : 234UP
Total stops in old data : 48
Total stops in new data : 48
Stop did not match : Inderlok Metro Station & Inder Lok
Stop did not match : Inderlok & Inder Lok Metro Station
Stop did not match : Khajuri & Khazoori
Stop did not match : Yamuna Vihar Brij Puri & C Block Yamuna Vihar
Route : 235DOWN
Total stops in old data : 42
Total stops in new data : 45
Stop did not match : GTB Hospital Xing & Gagan Caname Xing
Stop did not match : Hardev Puri & Bank Colony
Stop did not match : Jyoti Colony & NND
Stop did not match : Chhajjupur & Ashok Nager
Stop did not match : Shiv Mandir & L.R. Xing
Stop did not match : Babar Pur Ext & Jyoti Colony
Stop did not match : Krishna Marg & Chajju Pur 
Stop did not match : Vijay Park & Shiv Mandir
Stop did not match : C-4 Yamuna V X & Babar Pur Extn
Stop did not match : 

Route : 254UP
Total stops in old data : 48
Total stops in new data : 48
Stop did not match : Yamuna Vihar Brijpuri & C-Block Yamuna Vihar
Stop did not match : Khajuri & Khajoori
Stop did not match : Mukand Pur More & Mukand Pur Xing
Stop did not match : Mangolpur School & Mangol Puri Police Line
Stop did not match : Mangol Puri B Block / Vidya Vihar & B-Block Mangol Puri Ring Road
Stop did not match : Nangloi & Jawala Puri
Stop did not match : Nangloi JJ Colony Lokesh Cinema & Quamruddin Nagar
Route : 258DOWN
Total stops in old data : 26
Total stops in new data : 26
Stop did not match : Gurudwara Nanak Sar & Nanak Sar More
Stop did not match : Banarsi Daas Estate Timarpur & Banarasi Dass Estate
Route : 258SPLDOWN
Total stops in old data : 28
Total stops in new data : 28
Stop did not match : Tronika City Gate No 1 & Shaba Pur
Stop did not match : Sonia Vihar & Pusta No-4 Gatta Factory
Stop did not match : Gurudwara Nanak Sar & Nanaksar
Stop did not match : Gurudwara Nanak Sar & nanaksar

Route : 307UP
Total stops in old data : 24
Total stops in new data : 25
Stop did not match : Dichau Kalan Depot & Kamla Market
Stop did not match : Ajmeri Gate / Kamla Market & Vivekanand Marg Crossing
Stop did not match : Defence Colony & Rainy Well
Stop did not match : Ganesh Nagar & Shakkar pur School Block
Stop did not match : Mother Dairy & Ganesh Nagar
Stop did not match : Patparganj Xing & Mother Dairy
Stop did not match : P S Pandav Nagar & Samas Pur Village
Stop did not match : Shashi Garden Xing & Pandav Nagar Police Station
Stop did not match : ITI Khichiripur & Shashi Garden More
Stop did not match : Kotla Village & ITI Khichiri Pur
Stop did not match : New Rajdhani Enclave & Kotla Village
Stop did not match : KarKarDooma Crossing & Trilok Puri 13-Block
Stop did not match : Hassanpur Depot & Trilok Puri Blk-26 (Gurudwara)
Stop did not match : None & Trilok Pur 27 /31 Block
Route : 309DOWN
Total stops in old data : 34
Total stops in new data : 34
Stop did not match : Shashi 

Route : 335UP
Total stops in old data : 52
Total stops in new data : 52
Stop did not match : Delhi Secretariat & Rainy Well
Stop did not match : Shakarpur Crossing & Shakarpur X ing
Stop did not match : Bhor Garh & Swashthya Vihar
Stop did not match : East Azad Nagar & Jhar Khandi
Stop did not match : Jharkhandi & East Azad Nagar
Stop did not match : Hardev Puri & L.R. Xing
Stop did not match : GTB Hospital Xing & Nand Nagri Depot
Route : 336DOWN
Total stops in old data : 58
Total stops in new data : 79
Stop did not match : Preet Vihar & Swasthya Vihar
Stop did not match : Delhi Secretariat & Rainy Well
Stop did not match : Indian AirLines/Priya Cinema & Indian Air Lines Colony Vasant
Stop did not match : None & Railway Bridge Ring Road
Stop did not match : None & Pragati Maidan
Stop did not match : None & National Stadium
Stop did not match : None & High Court
Stop did not match : None & National Art Galery
Stop did not match : None & Shahajahan Road
Stop did not match : None & Kota H

Route : 348UP
Total stops in old data : 38
Total stops in new data : 38
Stop did not match : G P O Kasmiri Gate & GPO Kashmere Gate
Stop did not match : Kondali Old & Kondli Crossing
Stop did not match : Mayur Vihar Phase III A1 Block & Mayur Vihar Ph-3 Pkt. A-1
Stop did not match : Khora Cly Corner & Khora Colony
Route : 349ADOWN
Total stops in old data : 22
Total stops in new data : 22
Stop did not match : Laxmi Nagar Metro Station & Shakkar pur School Block
Stop did not match : Defence Colony & Laxmi Nagar Xing
Stop did not match : Delhi Secretariat & Rainy Well
Stop did not match : ITO & Delhi Sect.
Route : 349AUP
Total stops in old data : 21
Total stops in new data : 21
Stop did not match : IP Power House / ITO Ring Road & Delhi Secretariat
Stop did not match : Delhi Secretariat & Rainy Well
Stop did not match : Defence Colony & Laxmi Nagar Xing
Stop did not match : Laxmi Nagar Metro Station & Shakkar pur School Block
Stop did not match : Vinod Nagar & EVND
Route : 349DOWN
Total s